# Processing VCFs to run SNAPP

***

## Introduction

The idea of this "mini tutorial" is to generate the nexus file needed to run [SNAPP](https://www.beast2.org/snapp/). First, activate the environment within which we've been working (check [this](https://github.com/mftorres/popgen_ces/blob/master/VCF_processing_20200428.ipynb) on how to create the environment). Second, install `vcftools`, a package similar to `bcftools` to filter VCFs.

```bash
conda activate popgen
conda install -c bioconda vcftools
```

Now, we will generate a VCF including only SNPs with quality higher than 30 (`--minQ 30`), with no missing information in 100% of the samples (`--max-missing 1`), a maximum number of 2 alleles (`--max-alleles 2`), and with no indels (`--remove-indels`).

```bash
vcftools --vcf input.vcf --minQ 30 --max-missing 1 --remove-indels --recode --recode-INFO-all --max-alleles 2 --out filtered
# we will see something like: After filtering, kept 1508974 out of a possible 14779833 Sites
```
SNAPP analyses are a bit computationally expensive and tend to include between 1000 and 2000 SNPs only. We need to filter randomly 1000 SNPs. We will assume that every SNP in the filtered VCF is of excellent quality and that the file contains one SNP per line. First, we pass the header from the VCF to the new file by grepping every line that starts with a '#'. Second, we grep the opposite pattern (every line that does not start with a '#') and pipe the result to `shuf`, a command that generates 1000 random permutations, and we concatenate (>>) those to the file. Finally, we use the `vcf2phylip.py` (from [here](https://github.com/edgardomortiz/vcf2phylip)) script to convert the filtered VCF to a nexus.

```bash
grep '^#' filtered.recode.vcf > filtered_1ksnps.recode.vcf
grep -v '#' filtered.recode.vcf | shuf -n 1000 >> filtered_1ksnps.recode.vcf
python vcf2phylip.py -i filtered_1ksnps.recode.vcf -n
```